In [1]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def trade_option(symbol, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    start_time = start_time.replace(hour=9, minute=0, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)
     
    url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+symbol
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    
    return ohlc



In [8]:
symbol = 'NIFTY'
days_before = 0
timeframe = 10
ohlc = trade_option(symbol, days_before, timeframe)

position = False
entry_price = None
position_type = None
total_profit_loss = 0
trades_taken = 0

# Assume 'ohlc' is your OHLC data DataFrame
ohlc['HA_Close'] = (ohlc['open'] + ohlc['high'] + ohlc['low'] + ohlc['close']) / 4
ha_open = ohlc['HA_Close'].shift(1)
ohlc['HA_Open'] = ha_open.values[0]
ohlc.loc[1:, 'HA_Open'] = ha_open.values[1:]
ohlc['HA_High'] = ohlc[['HA_Open', 'HA_Close', 'high']].max(axis=1)
ohlc['HA_Low'] = ohlc[['HA_Open', 'HA_Close', 'low']].min(axis=1)

for index, row in ohlc.iterrows():
    if index == 0:
        i = 1
    else:
        i = index
        
    #if row['HA_Close'] >= row['HA_Open'] and ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] and ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']:
    if row['HA_Close'] >= row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']):
        if position and position_type == 'short':
            position = False
            profit_loss = entry_price - row['open']
            total_profit_loss += profit_loss
            print(f"Exit Sell: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

        if not position:
            position = True
            entry_price = row['open']
            position_type = "long"
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    #if row['HA_Close'] < row['HA_Open'] and ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] and ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']:
    if row['HA_Close'] < row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']):
        if position and position_type == 'long':
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

        if not position:
            position = True
            entry_price = row['open']
            position_type = "short"
            trades_taken += 1
            print(f"Sell: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

# Print open trades
# if open_trade:
#     print("\nOpen Trades:")
#     print(f"Type: {open_trade['type']}, Entry Time: {unix_to_local_time(open_trade['entry_time'])}, Entry Price: {open_trade['entry_price']}")

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)} Total Options: {round(total_profit_loss*0.45,1)}")
if(symbol == "NIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*25)-(trades_taken*60)}")
if(symbol == "BANKNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*15)-(trades_taken*60)}")
if(symbol == "FINNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*40)-(trades_taken*60)}")
if(symbol == "NIFTYMIDSELECT"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*75)-(trades_taken*60)}")

Sell: Entry Time: 2024-05-13 09:35:00, Entry Price: 10825.3
Exit Sell: Time: 2024-05-13 10:15:00, Exit Price: 10805.3, P/L: 20.0
Long: Entry Time: 2024-05-13 10:15:00, Entry Price: 10805.3
Exit Long: Time: 2024-05-13 10:45:00, Exit Price: 10769.75, P/L: -35.54999999999927
Sell: Entry Time: 2024-05-13 10:45:00, Entry Price: 10769.75
Exit Sell: Time: 2024-05-13 11:05:00, Exit Price: 10784.8, P/L: -15.049999999999272
Long: Entry Time: 2024-05-13 11:05:00, Entry Price: 10784.8
Exit Long: Time: 2024-05-14 09:25:00, Exit Price: 11060.1, P/L: 275.3000000000011
Sell: Entry Time: 2024-05-14 09:25:00, Entry Price: 11060.1
Exit Sell: Time: 2024-05-14 10:45:00, Exit Price: 11061.25, P/L: -1.1499999999996362
Long: Entry Time: 2024-05-14 10:45:00, Entry Price: 11061.25
Exit Long: Time: 2024-05-14 11:35:00, Exit Price: 11068.3, P/L: 7.049999999999272
Sell: Entry Time: 2024-05-14 11:35:00, Entry Price: 11068.3
Exit Sell: Time: 2024-05-14 11:55:00, Exit Price: 11086.65, P/L: -18.350000000000364
Long: E

1. and (or) - high(TF) low(Order,Profit), low(TF) high(Order,Profit)
2. and and  - hight(TF,profit) low(order), low(TF,profit) high(order) 

In [31]:
def get_atm_strike(symbol, current_price, current_trend):
    if symbol == "NIFTY":
        increment = 50
    elif symbol == "BANKNIFTY":
        increment = 100
    else:
        raise BotException("Invalid symbol at get_atm_strike")

    if current_price is None:
        raise BotException("Current price is none in get_atm_strike")

    if current_trend == "CE":
        rounded_strike = ((current_price - increment -1) // increment) * increment
    elif current_trend == "PE":
        rounded_strike = ((current_price + increment - 1) // increment) * increment
    elif current_trend is None:
        rounded_strike = 0
    else:
        raise BotException("Invalid current_trend value in get_atm_strike")

    return rounded_strike

    
get_atm_strike("NIFTY",22260,"PE")

22300